In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import calmap
import folium

# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
# from pandas import plotting
# import imp
# imp.find_module('pandas')
# imp.find_module('plotting')

In [3]:
covid = pd.read_csv('../data/processed/covid.csv', parse_dates=['Date'])
covid.sample(6)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
38877,Nova Scotia,Canada,44.6820,-63.7443,2020-06-16,1061,62,0,999
1665,Macau,China,22.1667,113.5500,2020-01-28,7,0,0,7
1686,NaN,Cyprus,35.1264,33.4299,2020-01-28,0,0,0,0
5448,NaN,Honduras,15.2000,-86.2419,2020-02-11,0,0,0,0
3654,NaN,Singapore,1.2833,103.8333,2020-02-04,24,0,0,24
18865,Yukon,Canada,64.2823,-135.0000,2020-04-01,5,0,0,5


In [4]:
covid.tail(7)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
53991,NaN,South Sudan,6.877000,31.307000,2020-08-11,2472,47,1175,1250
53992,NaN,Western Sahara,24.215500,-12.885800,2020-08-11,10,1,8,1
53993,NaN,Sao Tome and Principe,0.186400,6.613100,2020-08-11,881,15,804,62
53994,NaN,Yemen,15.552727,48.516388,2020-08-11,1831,523,915,393
53995,NaN,Comoros,-11.645500,43.333300,2020-08-11,399,7,379,13
53996,NaN,Tajikistan,38.861000,71.276100,2020-08-11,7871,63,6653,1155
53997,NaN,Lesotho,-29.610000,28.233600,2020-08-11,781,24,175,582


In [5]:
myDict = {"name": "John", "country": "Norway"}
mySeparator = "TEST"

x = mySeparator.join(myDict)
x

'nameTESTcountry'

# The daily reported data is accumulated.

In [6]:
covid[covid['Country/Region']=='Ukraine'].tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
52883,NaN,Ukraine,48.3794,31.1656,2020-08-07,80018,1879,44369,33770
53149,NaN,Ukraine,48.3794,31.1656,2020-08-08,81534,1906,45006,34622
53415,NaN,Ukraine,48.3794,31.1656,2020-08-09,82767,1925,45335,35507
53681,NaN,Ukraine,48.3794,31.1656,2020-08-10,83812,1950,45723,36139
53947,NaN,Ukraine,48.3794,31.1656,2020-08-11,85023,1979,46313,36731


In [7]:
covid_China = covid[covid['Country/Region']=='China']

In [8]:
# Since some countries have data reported for each individual state/province, it is necessary to group by country
covid = covid.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

In [9]:
covid

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0,0
1,2020-01-22,Albania,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0
4,2020-01-22,Angola,0,0,0,0
...,...,...,...,...,...,...
38159,2020-08-11,West Bank and Gaza,14875,104,8181,6590
38160,2020-08-11,Western Sahara,10,1,8,1
38161,2020-08-11,Yemen,1831,523,915,393
38162,2020-08-11,Zambia,8275,241,7004,1030


In [10]:
grouped = covid.groupby(['Country/Region', 'Date'])['Confirmed', 'Deaths', 'Recovered'].sum()
new_cases = grouped.diff().reset_index()
mask = new_cases['Country/Region'] != new_cases['Country/Region'].shift(1)
new_cases.loc[mask, ['Confirmed', 'Deaths', 'Recovered'] ] = np.nan

In [11]:
# renaming columns
new_cases.columns = ['Country/Region', 'Date', 'New Cases', 'New Deaths', 'New Recovered']

new_cases = new_cases[['Country/Region', 'Date', 'New Cases', 'New Deaths', 'New Recovered']]

covid = pd.merge(covid, new_cases, how='left', on=['Country/Region', 'Date'])

covid = covid.fillna(0)

# fixing data types
cols = ['New Cases', 'New Deaths', 'New Recovered']
covid[cols] = covid[cols].astype('int')

covid['New Cases'] = covid['New Cases'].apply(lambda x: 0 if x<0 else x)

covid.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0
1,2020-01-22,Albania,0,0,0,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0,0,0,0
4,2020-01-22,Angola,0,0,0,0,0,0,0


In [12]:
covid[covid['Country/Region']=='United States']

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered
178,2020-01-22,United States,1,0,0,1,0,0,0
366,2020-01-23,United States,1,0,0,1,0,0,0
554,2020-01-24,United States,2,0,0,2,1,0,0
742,2020-01-25,United States,2,0,0,2,0,0,0
930,2020-01-26,United States,5,0,0,5,3,0,0
...,...,...,...,...,...,...,...,...,...
37402,2020-08-07,United States,4941755,161347,1623870,3156538,58173,1243,25246
37590,2020-08-08,United States,4997929,162423,1643118,3192388,56174,1076,19248
37778,2020-08-09,United States,5044864,162938,1656864,3225062,46935,515,13746
37966,2020-08-10,United States,5094400,163463,1670755,3260182,49536,525,13891


In [13]:
world_daily = covid.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New Cases'].sum().reset_index()

world_daily['Death Rate'] = round(world_daily['Deaths']/world_daily['Confirmed']*100, 2)
world_daily['Recovery'] = round(world_daily['Recovered']/world_daily['Confirmed']*100, 2)
world_daily['Death/Recovered'] = round(world_daily['Deaths']/world_daily['Recovered']*100, 2)

# number of countries exposed to covid-19
world_daily['Countries'] = covid[covid['Confirmed']!=0].groupby('Date')['Country/Region'].unique().apply(len).values

cols = ['Death Rate', 'Recovery', 'Death/Recovered']
world_daily[cols] = world_daily[cols].fillna(0)

In [14]:
world_daily.tail()

,Date,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered,Countries
198,2020-08-07,19378036,721324,11630040,7026672,280887,3.72,60.02,6.20,188
199,2020-08-08,19637506,726781,11831070,7079655,259470,3.70,60.25,6.14,188
200,2020-08-09,19861683,731326,12007598,7122759,224179,3.68,60.46,6.09,188
201,2020-08-10,20089624,736191,12169625,7183808,227941,3.66,60.58,6.05,188
202,2020-08-11,20343040,742607,12474092,7126341,253417,3.65,61.32,5.95,188


In [15]:
countries = covid[covid['Date']==max(covid['Date'])].reset_index(drop=True).drop('Date', axis=1)

countries = countries.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New Cases'].sum().reset_index()

countries['Death Rate'] = round((countries['Deaths']/countries['Confirmed'])*100, 2)
countries['Recovery'] = round((countries['Recovered']/countries['Confirmed'])*100, 2)
countries['Death/Recovered'] = round((countries['Deaths']/countries['Recovered'])*100, 2)

cols = ['Death Rate', 'Recovery', 'Death/Recovered']
countries[cols] = countries[cols].fillna(0)

countries.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
0,Afghanistan,37269,1344,26415,9510,107,3.61,70.88,5.09
1,Albania,6676,205,3480,2991,140,3.07,52.13,5.89
2,Algeria,36204,1322,25263,9619,492,3.65,69.78,5.23
3,Andorra,963,52,839,72,0,5.40,87.12,6.20
4,Angola,1735,80,575,1080,56,4.61,33.14,13.91


In [16]:
countries.sort_values(by=['Death Rate'], ascending=False).head(10)

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
185,Yemen,1831,523,915,393,0,28.56,49.97,57.16
103,MS Zaandam,9,2,0,7,0,22.22,0.00,inf
177,United Kingdom,313394,46611,1464,265319,2,14.87,0.47,3183.81
85,Italy,251237,35215,202461,13561,412,14.02,80.59,17.39
16,Belgium,75008,9885,17841,47282,388,13.18,23.79,55.41
77,Hungary,4746,605,3527,614,15,12.75,74.32,17.15
62,France,239355,30328,82971,126056,6,12.67,34.66,36.55
112,Mexico,492522,53929,397278,41315,6686,10.95,80.66,13.57
121,Netherlands,60391,6162,244,53985,333,10.20,0.40,2525.41
184,Western Sahara,10,1,8,1,0,10.00,80.00,12.50


In [17]:
countries.sort_values(by=['Deaths'], ascending=False).head(10)

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
178,United States,5141208,164537,1714960,3261711,46808,3.20,33.36,9.59
23,Brazil,3109630,103026,2449338,557266,52160,3.31,78.77,4.21
112,Mexico,492522,53929,397278,41315,6686,10.95,80.66,13.57
177,United Kingdom,313394,46611,1464,265319,2,14.87,0.47,3183.81
79,India,2329638,46091,1639599,643948,60963,1.98,70.38,2.81
85,Italy,251237,35215,202461,13561,412,14.02,80.59,17.39
62,France,239355,30328,82971,126056,6,12.67,34.66,36.55
158,Spain,326612,28581,150376,147655,3632,8.75,46.04,19.01
133,Peru,489680,21501,329404,138775,6547,4.39,67.27,6.53
81,Iran,331189,18800,288620,23769,2345,5.68,87.15,6.51


In [18]:
countries.sort_values(by=['Recovery'], ascending=False).head(25)

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
75,Holy See,12,0,12,0,0,0.00,100.00,0.00
141,Saint Kitts and Nevis,17,0,17,0,0,0.00,100.00,0.00
142,Saint Lucia,25,0,25,0,0,0.00,100.00,0.00
50,Dominica,18,0,18,0,0,0.00,100.00,0.00
149,Seychelles,127,0,126,1,1,0.00,99.21,0.00
100,Liechtenstein,89,1,87,1,0,1.12,97.75,1.15
122,New Zealand,1570,22,1526,22,0,1.40,97.20,1.44
24,Brunei,142,3,138,1,0,2.11,97.18,2.17
111,Mauritius,344,10,334,0,0,2.91,97.09,2.99
137,Qatar,113646,188,110324,3134,384,0.17,97.08,0.17


In [19]:
countries.sort_values(by=['Recovery'], ascending=False).head(25)

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
75,Holy See,12,0,12,0,0,0.00,100.00,0.00
141,Saint Kitts and Nevis,17,0,17,0,0,0.00,100.00,0.00
142,Saint Lucia,25,0,25,0,0,0.00,100.00,0.00
50,Dominica,18,0,18,0,0,0.00,100.00,0.00
149,Seychelles,127,0,126,1,1,0.00,99.21,0.00
100,Liechtenstein,89,1,87,1,0,1.12,97.75,1.15
122,New Zealand,1570,22,1526,22,0,1.40,97.20,1.44
24,Brunei,142,3,138,1,0,2.11,97.18,2.17
111,Mauritius,344,10,334,0,0,2.91,97.09,2.99
137,Qatar,113646,188,110324,3134,384,0.17,97.08,0.17


In [20]:
countries.sort_values(by=['Active'], ascending=False).head(25)

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
178,United States,5141208,164537,1714960,3261711,46808,3.20,33.36,9.59
79,India,2329638,46091,1639599,643948,60963,1.98,70.38,2.81
23,Brazil,3109630,103026,2449338,557266,52160,3.31,78.77,4.21
177,United Kingdom,313394,46611,1464,265319,2,14.87,0.47,3183.81
139,Russia,895691,15103,701796,178792,4892,1.69,78.35,2.15
37,Colombia,410453,13475,230427,166551,12830,3.28,56.14,5.85
158,Spain,326612,28581,150376,147655,3632,8.75,46.04,19.01
133,Peru,489680,21501,329404,138775,6547,4.39,67.27,6.53
155,South Africa,566109,10751,426125,129233,2511,1.90,75.27,2.52
62,France,239355,30328,82971,126056,6,12.67,34.66,36.55


In [21]:
countries.sort_values(by=['New Cases'], ascending=False).head(25)

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
79,India,2329638,46091,1639599,643948,60963,1.98,70.38,2.81
23,Brazil,3109630,103026,2449338,557266,52160,3.31,78.77,4.21
178,United States,5141208,164537,1714960,3261711,46808,3.20,33.36,9.59
37,Colombia,410453,13475,230427,166551,12830,3.28,56.14,5.85
6,Argentina,260911,5004,181389,74518,7043,1.92,69.52,2.76
112,Mexico,492522,53929,397278,41315,6686,10.95,80.66,13.57
133,Peru,489680,21501,329404,138775,6547,4.39,67.27,6.53
139,Russia,895691,15103,701796,178792,4892,1.69,78.35,2.15
158,Spain,326612,28581,150376,147655,3632,8.75,46.04,19.01
82,Iraq,156995,5531,112102,39362,3396,3.52,71.40,4.93


In [23]:
population = pd.read_csv('../data/raw/API_SP.POP.TOTL_DS2_en_csv_v2_988606.csv')
population

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,61032.0,62149.0,64622.0,68235.0,72504.0,76700.0,80324.0,83200.0,85451.0,87277.0,89005.0,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,10174836.0,10399926.0,10637063.0,10893776.0,11173642.0,11475445.0,11791215.0,12108963.0,12412950.0,12689160.0,12943093.0,13171306.0,13341198.0,13411056.0,13356511.0,13171673.0,12882528.0,12537730.0,12204292.0,11938208.0,11736179.0,11604534.0,11618005.0,11868877.0,12412308.0,13299017.0,14485546.0,15816603.0,17075727.0,18110657.0,18853437.0,19357126.0,19737765.0,20170844.0,20779953.0,21606988.0,22600770.0,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,5781214.0,5774243.0,5771652.0,5803254.0,5890365.0,6040777.0,6248552.0,6496962.0,6761380.0,7024000.0,7279509.0,7533735.0,7790707.0,8058067.0,8341289.0,8640446.0,8952950.0,9278096.0,9614754.0,9961997.0,10320111.0,10689250.0,11068050.0,11454777.0,11848386.0,12248901.0,12657366.0,13075049.0,13503747.0,13945206.0,14400719.0,14871570.0,15359601.0,15866869.0,16395473.0,16945753.0,17519417.0,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,3227943.0,3286542.0,3266790.0,3247039.0,3227287.0,3207536.0,3187784.0,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,52448.0,54509.0,56671.0,58888.0,60971.0,62677.0,63850.0,64360.0,64327.0,64142.0,64370.0,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,1106000.0,1135000.0,1163000.0,1191000.0,1219000.0,1247000.0,1278000.0,1308000.0,1339000.0,1369000.0,1400000.0,1430000.0,1460000.0,1491000.0,1521000.0,1552000.0,1582000.0,1614000.0,1647000.0,1682000.0,1717000.0,1753

In [24]:
population = population[['Country Name', '2018']]
population.columns = ['Country/Region', 'Population']
population['Population'] = population['Population'].fillna(0)
population['Population'] = population['Population'].astype('int')
population

,Country/Region,Population
0,Aruba,105845
1,Afghanistan,37172386
2,Angola,30809762
3,Albania,2866376
4,Andorra,77006
...,...,...
259,Kosovo,1845300
260,"Yemen, Rep.",28498687
261,South Africa,57779622
262,Zambia,17351822


# Timeline

In [25]:
fig_1 = px.line(world_daily, x="Date", y="Confirmed", color_discrete_sequence = [act])
fig_2 = px.line(world_daily, x="Date", y="Deaths", color_discrete_sequence = [dth])

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('Confirmed cases', 'Deaths reported'))

fig.add_trace(fig_1['data'][0], row=1, col=1)
fig.add_trace(fig_2['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [26]:
'''
'''

'\n'

In [27]:
fig = px.line(covid, x="Date", y="Deaths", color="Country/Region", line_group="Country/Region", hover_name="Country/Region",
        line_shape="spline", render_mode="svg")
fig.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [28]:
confirmed_top12 = countries.sort_values(by='Confirmed', ascending=False).head(12)
confirmed_top12

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
178,United States,5141208,164537,1714960,3261711,46808,3.20,33.36,9.59
23,Brazil,3109630,103026,2449338,557266,52160,3.31,78.77,4.21
79,India,2329638,46091,1639599,643948,60963,1.98,70.38,2.81
139,Russia,895691,15103,701796,178792,4892,1.69,78.35,2.15
155,South Africa,566109,10751,426125,129233,2511,1.90,75.27,2.52
112,Mexico,492522,53929,397278,41315,6686,10.95,80.66,13.57
133,Peru,489680,21501,329404,138775,6547,4.39,67.27,6.53
37,Colombia,410453,13475,230427,166551,12830,3.28,56.14,5.85
35,Chile,376616,10178,349541,16897,1572,2.70,92.81,2.91
81,Iran,331189,18800,288620,23769,2345,5.68,87.15,6.51


In [29]:
confirmed_top12['Country/Region']

178     United States
23             Brazil
79              India
139            Russia
155      South Africa
112            Mexico
133              Peru
37           Colombia
35              Chile
81               Iran
158             Spain
177    United Kingdom
Name: Country/Region, dtype: object

In [30]:
mask = covid['Country/Region'].isin(list(confirmed_top12['Country/Region']))
top12_countries = covid[mask]
top12_countries

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered
23,2020-01-22,Brazil,0,0,0,0,0,0,0
35,2020-01-22,Chile,0,0,0,0,0,0,0
37,2020-01-22,Colombia,0,0,0,0,0,0,0
79,2020-01-22,India,0,0,0,0,0,0,0
81,2020-01-22,Iran,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
38115,2020-08-11,Russia,895691,15103,701796,178792,4892,130,6479
38131,2020-08-11,South Africa,566109,10751,426125,129233,2511,130,8925
38134,2020-08-11,Spain,326612,28581,150376,147655,3632,5,0
38153,2020-08-11,United Kingdom,313394,46611,1464,265319,2,0,1


In [31]:
fig = px.line(top12_countries, x="Date", y="Confirmed", color="Country/Region", line_group="Country/Region", hover_name="Country/Region",
        line_shape="spline", render_mode="svg")
fig.show()

In [76]:
fig = px.line(top12_countries, x="Date", y="Deaths", color="Country/Region", line_group="Country/Region", hover_name="Country/Region",
        line_shape="spline", render_mode="svg", title='Cumulated Deaths')
fig.show()

In [33]:
fig = px.line(top12_countries, x="Date", y="Active", color="Country/Region", line_group="Country/Region", hover_name="Country/Region",
        line_shape="spline", render_mode="svg")
fig.show()

In [34]:
fig = px.line(top12_countries, x="Date", y="New Cases", color="Country/Region", line_group="Country/Region", hover_name="Country/Region",
        line_shape="spline", render_mode="svg")
fig.show()

In [35]:
fig = px.line(top12_countries, x="Date", y="New Deaths", color="Country/Region", line_group="Country/Region", hover_name="Country/Region",
        line_shape="spline", render_mode="svg")
fig.show()

In [36]:
covid_us = covid[covid['Country/Region']=='United States']
covid_us

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered
178,2020-01-22,United States,1,0,0,1,0,0,0
366,2020-01-23,United States,1,0,0,1,0,0,0
554,2020-01-24,United States,2,0,0,2,1,0,0
742,2020-01-25,United States,2,0,0,2,0,0,0
930,2020-01-26,United States,5,0,0,5,3,0,0
...,...,...,...,...,...,...,...,...,...
37402,2020-08-07,United States,4941755,161347,1623870,3156538,58173,1243,25246
37590,2020-08-08,United States,4997929,162423,1643118,3192388,56174,1076,19248
37778,2020-08-09,United States,5044864,162938,1656864,3225062,46935,515,13746
37966,2020-08-10,United States,5094400,163463,1670755,3260182,49536,525,13891


In [37]:
confirmed_top12

,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,Death Rate,Recovery,Death/Recovered
178,United States,5141208,164537,1714960,3261711,46808,3.20,33.36,9.59
23,Brazil,3109630,103026,2449338,557266,52160,3.31,78.77,4.21
79,India,2329638,46091,1639599,643948,60963,1.98,70.38,2.81
139,Russia,895691,15103,701796,178792,4892,1.69,78.35,2.15
155,South Africa,566109,10751,426125,129233,2511,1.90,75.27,2.52
112,Mexico,492522,53929,397278,41315,6686,10.95,80.66,13.57
133,Peru,489680,21501,329404,138775,6547,4.39,67.27,6.53
37,Colombia,410453,13475,230427,166551,12830,3.28,56.14,5.85
35,Chile,376616,10178,349541,16897,1572,2.70,92.81,2.91
81,Iran,331189,18800,288620,23769,2345,5.68,87.15,6.51


In [38]:
df_dict = {}
for it in list(confirmed_top12['Country/Region']):    
    df_dict[it] = covid[covid['Country/Region']==it]
    df_dict[it]['Deaths_7day_Mean'] = df_dict[it]['Deaths'].rolling(window=7).mean()

In [39]:
df = df_dict['United States']
df

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered,Deaths_7day_Mean
178,2020-01-22,United States,1,0,0,1,0,0,0,NaN
366,2020-01-23,United States,1,0,0,1,0,0,0,NaN
554,2020-01-24,United States,2,0,0,2,1,0,0,NaN
742,2020-01-25,United States,2,0,0,2,0,0,0,NaN
930,2020-01-26,United States,5,0,0,5,3,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
37402,2020-08-07,United States,4941755,161347,1623870,3156538,58173,1243,25246,157802.142857
37590,2020-08-08,United States,4997929,162423,1643118,3192388,56174,1076,19248,158839.857143
37778,2020-08-09,United States,5044864,162938,1656864,3225062,46935,515,13746,159893.142857
37966,2020-08-10,United States,5094400,163463,1670755,3260182,49536,525,13891,160944.428571


In [40]:
df_dict.pop('United States')
df_dict.keys()

dict_keys(['Brazil', 'India', 'Russia', 'South Africa', 'Mexico', 'Peru', 'Colombia', 'Chile', 'Iran', 'Spain', 'United Kingdom'])

In [41]:
for key in df_dict:    
    df = pd.concat([df, df_dict[key]])

In [42]:
df

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered,Deaths_7day_Mean
178,2020-01-22,United States,1,0,0,1,0,0,0,NaN
366,2020-01-23,United States,1,0,0,1,0,0,0,NaN
554,2020-01-24,United States,2,0,0,2,1,0,0,NaN
742,2020-01-25,United States,2,0,0,2,0,0,0,NaN
930,2020-01-26,United States,5,0,0,5,3,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
37401,2020-08-07,United Kingdom,310696,46596,1447,262653,900,98,0,46388.000000
37589,2020-08-08,United Kingdom,311461,46651,1449,263361,765,55,2,46441.285714
37777,2020-08-09,United Kingdom,312574,46659,1451,264464,1113,8,2,46494.571429
37965,2020-08-10,United Kingdom,313392,46611,1463,265318,818,-48,12,46539.714286


In [43]:
fig = px.line(df, 
              x="Date", 
              y="Deaths", 
              color="Country/Region", 
              line_group="Country/Region", 
              hover_name="Country/Region",
              line_shape="spline", 
              render_mode="svg",
              title='Accumulated Number of Deaths')
fig.show()

In [44]:
fig = px.line(df, 
              x="Date", 
              y="Deaths_7day_Mean", 
              color="Country/Region", 
              line_group="Country/Region", 
              hover_name="Country/Region",
              line_shape="spline", 
              render_mode="svg",
              title='')
fig.show()

In [45]:
df_dict = {}
for it in list(confirmed_top12['Country/Region']):    
    df_dict[it] = covid[covid['Country/Region']==it]
    df_dict[it]['New_Deaths_3day_Mean'] = df_dict[it]['New Deaths'].rolling(window=3).mean()

df = pd.DataFrame()

for key in df_dict:    
    df = pd.concat([df, df_dict[key]])
    
fig = px.line(df, 
              x="Date", 
              y="New_Deaths_3day_Mean", 
              color="Country/Region", 
              line_group="Country/Region", 
              hover_name="Country/Region",
              line_shape="spline", 
              render_mode="svg",
              title='New Deaths Every 3 Days Top 12 Countries')
fig.show()

In [46]:
df_dict = {}
for it in list(confirmed_top12['Country/Region']):    
    df_dict[it] = covid[covid['Country/Region']==it]
    df_dict[it]['New_Deaths_7day_Mean'] = df_dict[it]['New Deaths'].rolling(window=7).mean()

df = pd.DataFrame()

for key in df_dict:    
    df = pd.concat([df, df_dict[key]])
    
fig = px.line(df, 
              x="Date", 
              y="New_Deaths_7day_Mean", 
              color="Country/Region", 
              line_group="Country/Region", 
              hover_name="Country/Region",
              line_shape="spline", 
              render_mode="svg",
              title='New Deaths Every Week 12 Top Countries')
fig.show()

In [47]:
# 21 day average
df_dict = {}
for it in list(confirmed_top12['Country/Region']):    
    df_dict[it] = covid[covid['Country/Region']==it]
    df_dict[it]['New_Deaths_7day_Mean'] = df_dict[it]['New Deaths'].rolling(window=21).mean()
    
df = pd.DataFrame()

for key in df_dict:    
    df = pd.concat([df, df_dict[key]])
    
fig = px.line(df, 
              x="Date", 
              y="New_Deaths_7day_Mean", 
              color="Country/Region", 
              line_group="Country/Region", 
              hover_name="Country/Region",
              line_shape="spline", render_mode="svg",
              title='New Deaths Every 3 Weeks for Top 12 Countries')
fig.show()

In [77]:
covid = pd.read_csv('../data/processed/covid.csv')
covid[covid['Date']==max(covid['Date'])]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
53200,NaN,Afghanistan,33.939110,67.709953,8/9/20,37054,1312,25960,9782
53201,NaN,Albania,41.153300,20.168300,8/9/20,6411,199,3342,2870
53202,NaN,Algeria,28.033900,1.659600,8/9/20,35160,1302,24506,9352
53203,NaN,Andorra,42.506300,1.521800,8/9/20,955,52,839,64
53204,NaN,Angola,-11.202700,17.873900,8/9/20,1672,75,567,1030
...,...,...,...,...,...,...,...,...,...
53461,NaN,Sao Tome and Principe,0.186400,6.613100,8/9/20,878,15,800,63
53462,NaN,Yemen,15.552727,48.516388,8/9/20,1804,515,913,376
53463,NaN,Comoros,-11.645500,43.333300,8/9/20,399,7,369,23
53464,NaN,Tajikistan,38.861000,71.276100,8/9/20,7745,62,6484,1199
